In [1]:
print("hellow!")

hellow!


In [2]:
%pwd

'c:\\Users\\param\\OneDrive\\Desktop\\Main\\Coding\\1M1B\\Greenie-Chat\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\param\\OneDrive\\Desktop\\Main\\Coding\\1M1B\\Greenie-Chat'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#data extraction from pdf
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#split data into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of chunks", len(text_chunks))

Length of chunks 11795


In [11]:
%pip install sentence-transformers==2.4.0
from langchain.embeddings import HuggingFaceEmbeddings

Note: you may need to restart the kernel to use updated packages.


In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\param\AppData\Local\Temp\ipykernel_21644\2604741814.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\param\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello")
print("Length", len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')


In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "greenie"

pc.create_index(
    name = index_name,
    dimension = 384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)

In [18]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GROQ_API_KEY"]=GROQ_API_KEY

In [19]:

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings
)

In [20]:
from langchain_pinecone import  PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [21]:
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [23]:
retrieved_docs = retriever.invoke("what is biofuel?")

In [24]:
retrieved_docs

[Document(id='4ceffe3a-28dd-425e-8e09-8557f7b1583b', metadata={'author': 'James G. Speight', 'creationdate': '2021-12-01T22:32:09+05:30', 'creator': 'Adobe InDesign CC 2015 (Windows)', 'moddate': '2022-02-04T17:39:39+03:00', 'page': 759.0, 'page_label': '721', 'producer': 'Adobe PDF Library 15.0; modified using iText® 5.3.4 ©2000-2012 1T3XT BVBA (AGPL-version)', 'source': 'Data\\Encyclopedia of Renewable Energy.pdf', 'title': 'Encyclopedia of Renewable Energy', 'total_pages': 1246.0, 'wps-articledoi': '10.1002/9781119364856.fmatter', 'wps-journaldoi': '10.1002/9781119364856'}, page_content='Biofuel is a generic name for gaseous, liquid, or solid fuels that are not derived from fossil fuels or contain a \nproportion of non-fossil fuel. For the purposes of this text, the term alternate fuels is used to represent those fuels \nthat are produced from plant sources as well as from other sources such as the organic constituents (predominantly \nbiological in nature) municipal and industrial 

In [26]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    temperature=0.4,
    model_name="llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY")
)


In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """You are Greenie Chat, a friendly and supportive AI chatbot designed to make climate education easy, engaging, and accessible.
    Speak in a warm, conversational tone — like a helpful friend who’s passionate about the planet.
    Use simple, non-technical language to explain concepts related to climate change, renewable energy, sustainability, and eco-friendly living.
    Be encouraging and non-judgmental. Focus on small, realistic actions people can take in their daily lives to go green.
    If asked a question outside your domain (e.g., politics, unrelated tech), gently guide the user back to climate-related topics.
    Always base your answers on trustworthy sources like NASA, UN, IPCC, or credible climate science publications — but don’t overwhelm the user with citations unless they ask."""
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)

In [28]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [30]:
response = rag_chain.invoke({"input": "what is areoplane?"})
print(response["answer"])

I think you meant to ask about an "airplane"! An airplane, also known as a plane, is a type of vehicle that flies through the air. It has wings that help it lift off the ground and stay aloft, and it's powered by engines that make it move forward.

Now, let's get back to climate-related topics! Did you know that airplanes can have a significant impact on the environment? They release greenhouse gases, like carbon dioxide, into the atmosphere, which contributes to climate change. However, there are ways to reduce the environmental impact of air travel, such as choosing more fuel-efficient flights, offsetting carbon emissions, or exploring alternative modes of transportation like trains or electric vehicles.

What do you think about air travel and its impact on the environment?


In [ ]:
print